# Nitish Prabhu Kota
## Student ID: 1001738851

### Write a python program that fetches movie information for the top 500 most popular movies from Metacritics. On this websites, there is an option to show the top movies. 
### On Metacritics, it is called “Movies of All Time”

### You will first write python script that collect the movie information for the top 500 movies from each website and store them in a comma separated file (called [your name]_movies.csv). 

### In addition to the csv file, the data should also be stored in a SQLite database called MovieInfoDatabase in the directory that your Jupyter Notebook code will be executed from. The MovieInfoDatabase should have a table called MovieInfoTable. 


In [1]:
# Imports
from bs4 import BeautifulSoup
import ssl
import requests
import sqlite3
import csv

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
#we need to use a user agent here, because metacritics does not accept bots
#we are using a tested go-to user agent here, it can be found online
#read more about user agents here:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

#### Loop through all the 5 pages of the movies, and store the names of the movies in the list movie_names. In addition obtain the movie links and use them to navigate to the movie page. we will have a details section for each movie, so fetch the link of details section for each movie and store them in the list detail_links. In ths process we use beautiful soup to extract the page ements to obtain details like movie name and links

In [5]:
#100 movie each page
#we need 5 pages
movie_names = []
movie_links = []
detail_links = []

directors_list = []
p_cast_list = []
cast_list = []
print('This process will need a minute to complete. Please wait until a completed message is displayed below')
for i in range(5):
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')
    for movie in container:
        name = movie.find('h3').text
        movie_names.append(name)
        tag = movie.find('a', class_ = 'title')
        link = tag.get('href', None)
        movie_links.append('https://www.metacritic.com'+link)

for movie_link in movie_links:
    page = requests.get(movie_link, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup('div', class_= 'see_all pad_btm1')
    for line in div:
        hyperlink = line.find('a')
        url = hyperlink.get('href', None)
        detail_links.append('https://www.metacritic.com'+ url)
        
# Links to the details section pages of all the movies are stored here
# print(detail_links)
print('Detail links of all the movies are stored in the list successfully. Please proceed with the next cell.')

This process will need a minute to complete. Please wait until a completed message is displayed below
Detail links of all the movies are stored in the list successfully. Please proceed with the next cell.


#### Now we use beautiful soup to extract directors table, cast table and principle cast table details for wach of the 500 movies, these details are stored in 3 arrays directors_list, p_cast_list and cast_list (all having lengths 500 i.e. 500 movies)


In [6]:
directors_list = []
p_cast_list = []
cast_list = []
print('This process will need around 2 minutes to complete. Please wait until a completed message is displayed below')
for i,detail_link in enumerate(detail_links):
    url = detail_link
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    apostrophe="'s"
    if movie_names[i].endswith("s"):
        apostrophe="'"
        
    director_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + apostrophe + " Director Credits")
#     print(director_table)
#     print(movie_names[i])
    if (director_table):
        director_line = director_table.find_all('td', class_ = 'person')
    #     print(director_line)
        directors = list()
        for line in director_line:
            hyperlink = line.find('a')
            director = hyperlink.get_text()
            directors.append(director.strip())
        directors_list.append(directors)
    else:
#         print("------------------------No director------------------------------")
        directors_list.append([])
    
    
    p_cast_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + apostrophe + " Principal Cast Credits")
#     print(p_cast_table)
    if (p_cast_table):
        p_cast_line = p_cast_table.find_all('td', class_ = 'person')
    #     print(p_cast_line)
        p_casts = list()
        for line in p_cast_line:
            hyperlink = line.find('a')
            p_cast = hyperlink.get_text()
            p_casts.append(p_cast.strip())
        p_cast_list.append(p_casts)
    else:
        p_cast_list.append([])
    
    
    cast_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + apostrophe + " Cast Credits")
#     print(cast_table)
    if (cast_table):
        cast_line = cast_table.find_all('td', class_ = 'person')
    #     print(cast_line)
        casts = list()
        for line in cast_line:
            hyperlink = line.find('a')
            cast = hyperlink.get_text()
            casts.append(cast.strip())
        cast_list.append(casts)
    else:
        cast_list.append([])
        
print('directors_list, p_cast_list and cast_list are fetched and stored in the arrays directors_list, p_cast_list and cast_list respectively')
print('Please proceed with the next cell.')
# print(directors_list)
# print(p_cast_list)
# print(cast_list)

This process will need around 2 minutes to complete. Please wait until a completed message is displayed below
Citizen Kane
The Godfather
Rear Window
Casablanca
Boyhood
Three Colors: Red
Vertigo
Notorious
Singin' in the Rain
City Lights
Moonlight
Intolerance
Pinocchio
Touch of Evil
The Lady Vanishes
The Treasure of the Sierra Madre
Pan's Labyrinth
Some Like It Hot
North by Northwest
Hoop Dreams
Rashomon
All About Eve
Jules and Jim
The Wild Bunch
My Left Foot
The Third Man
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb
Quo Vadis, Aida?
Gone with the Wind
4 Months, 3 Weeks and 2 Days
Psycho
Battleship Potemkin
A Streetcar Named Desire
American Graffiti
Dumbo
Roma
Ran
The Shop Around the Corner
12 Angry Men
Manchester by the Sea
Rosemary's Baby
The Maltese Falcon
12 Years a Slave
Killer of Sheep
Nashville
Ratatouille
Parasite
Rocks
Don't Look Now
The Grapes of Wrath
Children of Paradise (1945)
Mean Streets
The Lady Eve
Gravity
Fantasia
Spirited Away
Beauty and the Bea

The Duke of Burgundy
My Golden Days
Nebraska
What the Constitution Means to Me
A Tale of Two Cities
Force Majeure
Tampopo
Blow Out
Diane
The Magician
The Thin Man
Russian Ark
Point Blank
Lincoln
The Wolf House
An Elephant Sitting Still
Room
The Tillman Story
The Wind Will Carry Us
The Remains of the Day
sex, lies, and videotape
Dog Day Afternoon
directors_list, p_cast_list and cast_list are fetched and stored in the arrays directors_list, p_cast_list and cast_list respectively
Please proceed with the next cell.


In [8]:
# print(len(directors_list))
# print(len(p_cast_list))
# print(len(cast_list))

# print(directors_list)
# print(p_cast_list)
# print(cast_list)

#### Data is also stored in a SQLite database called MovieInfoDatabase in the directory that your Jupyter Notebook code will be executed from. The MovieInfoDatabase should have a table called MovieInfoTable.
#### A file with a name'MovieInfoDatabase.sqlite' is saved in the same directory as the .ipynb file
#### It has columns movie names, director list, principal cast list and the cat list

In [10]:
conn = sqlite3.connect('MovieInfoDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieInfoTable')
#CREATE TABLE + {table name} + (Column1 Constraint, Column2 Constraint...)
cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfoTable (
            movie_name TEXT NOT NULL, 
            directors_list TEXT, 
            p_cast_list TEXT, 
            cast_list TEXT)''')

# Save (commit) the changes
conn.commit()

# We can close the connection if we are done with it.
# We need to make sure all changes have been committed or they will be lost.
# conn.close()

for i in range(len(movie_names)):
    cur.execute('INSERT INTO MovieInfoTable (movie_name, directors_list, p_cast_list, cast_list) VALUES (?, ?, ?, ?)',
                ( movie_names[i], ', '.join(directors_list[i]), ', '.join(p_cast_list[i]), ', '.join(cast_list[i]))) 
# print('Movies:') 
# cur.execute('SELECT * FROM MovieInfoTable')
# for row in cur:
#     print(row) 
conn.commit()
conn.close()

#### python script that collect the movie information for the top 500 movies from each website and store them in a comma separated file (called 'NitishPrabhuKota_movies.csv' ). 
#### A file with a name'NitishPrabhuKota_movies.csv' is saved in the same directory as the .ipynb file


In [11]:
with open('NitishPrabhuKota_movies.csv', mode='w', newline='') as movie_file:
    movies_writer = csv.writer(movie_file, delimiter=',')
    movies_writer.writerow(['Movie Name', 'Director(s)', 'Principal Cast(s)', 'Cast(s)'])
    
    for i in range(len(movie_names)):
        movies_writer.writerow([movie_names[i], ', '.join(directors_list[i]), ', '.join(p_cast_list[i]), ', '.join(cast_list[i])])

In [12]:
# movie_names = list(map(lambda x:x.lower(), movie_names))
# temp_directors_list=[]
# temp_p_cast_list=[]
# temp_cast_list=[]

# for i in directors_list:
#     temp_directors_list.append(list(map(lambda x:x.lower(), i)))


# for i in p_cast_list:
#     temp_p_cast_list.append(list(map(lambda x:x.lower(), i)))
    

# for i in cast_list:
#     temp_cast_list.append(list(map(lambda x:x.lower(), i)))
    
# # p_cast_list = list(map(lambda x:x.lower(), p_cast_list))
# # cast_list = list(map(lambda x:x.lower(), cast_list))

# directors_list = temp_directors_list
# p_cast_list = temp_p_cast_list
# cast_list = temp_cast_list

# print(movie_names)
# print(directors_list)
# print(p_cast_list)
# print(cast_list)

### Next, from the movie information you have collected, extract 2 pieces of information: The director, and the cast (actors/actresses). Build a dictionary of the movies that contain these information. Arrange them in any way you prefer but make sure we can access the information we need at any time. 

#### Below is the structure of the dictionary which contains all the movies and their details. 
#### In this case movie name alone cant be used as a unique identifier since there may be few moves with the same names
#### I have used a tuple of movie name and director(s) as a composite key for the key attribute in the dictionary. Hence this dictionary will contain 500 movie and their details

In [ ]:
# dictionary_structure = { (movie_name_1, director_list_1): {(director_list_1): [cast_list_1 + principal_cast_list_1]},
#                          (movie_name_2, director_list_2): {(director_list_2): [cast_list_2 + principal_cast_list_2]},
#                           ... ,
#                           ... ,
#                          (movie_name_n, director_list_n): {(director_list_n): [cast_list_n + principal_cast_list_n]} }

In [14]:
movie_director = dict()
for i in range(len(detail_links)):
    director_cast = dict()
    director_cast[tuple(directors_list[i])] = list(set(p_cast_list[i] + cast_list[i]))
    
    res = tuple([movie_names[i]] + directors_list[i] )
    movie_director[res] = director_cast
                    
print(movie_director)
# print(len(movie_director))

{('Citizen Kane', 'Orson Welles'): {('Orson Welles',): ['Harry Shannon', 'Ray Collins', 'Erskine Sanford', 'Everett Sloane', 'Orson Welles', 'Paul Stewart', 'Philip Van Zandt', 'Dorothy Comingore', 'Joseph Cotten', 'Fortunio Bonanova', 'Ruth Warrick', 'Gus Schilling', 'William Alland', 'Georgia Backus', 'Agnes Moorehead', 'George Coulouris']}, ('The Godfather', 'Francis Ford Coppola'): {('Francis Ford Coppola',): ['Sterling Hayden', 'Rudy Bond', 'Marlon Brando', 'James Caan', 'John Marley', 'John Cazale', 'Gianni Russo', 'Richard S. Castellano', 'Robert Duvall', 'Diane Keaton', 'Richard Conte', 'Al Pacino', 'Talia Shire', 'Abe Vigoda', 'Al Lettieri']}, ('Rear Window', 'Alfred Hitchcock'): {('Alfred Hitchcock',): ['Grace Kelly', 'Rand Harper', 'Judith Evelyn', 'Jesslyn Fax', 'Ross Bagdasarian', 'Sara Berner', 'Irene Winston', 'Jerry Antes', 'Raymond Burr', 'Havis Davenport', 'Wendell Corey', 'Thelma Ritter', 'Georgine Darcy', 'James Stewart', 'Frank Cady']}, ('Casablanca', 'Michael Curt

#### The program prompts the use to provide the following as shown in example below:
#### Example: 
#### Which movie do you want to check?
#### input: Saving Private Ryan
#### What information about this movie do you want to check? (Choose director or cast)
#### 	input: Cast

####  The user is prompted repeatedly to enter the movie name correctly until the user provides teh righ name (case sensitivity is handled)
#### Based on input provided by the user, details of the movie are displayed below

In [15]:
movie_names_list_lower = list(map(lambda x:x.lower(), movie_names))
inp_movie = input('Which movie do you want to check? >').strip().lower()
while True:
    try:
        # Raise an exception if the movie is ot in the list
        if inp_movie not in movie_names_list_lower:
            raise
        else:
            break
    except:
        inp_movie = input('Please enter a valid movie name. Which movie do you want to check? >').strip().lower()

inp = input('What information about this movie do you want to check? (Choose director or cast) >').strip().lower()
while True:
    try:
        # Raise an exception if user enters wrong choice
        if inp not in ['director', 'cast']:
            raise
        else:
            break
    except:
        inp = input('Please enter a valid choice. (Choose director or cast) >').strip().lower()
        
for movie_key, movie_value in movie_director.items():
    if inp_movie.lower() == movie_key[0].lower():
        if inp == 'director':
            print('\nThe director of the movie "'+ inp_movie+ '" is/are:\n'+ ', '.join(movie_key[1:]))
            print('\n')
        elif inp =='cast':
            print('\nThe cast of the movie "'+ inp_movie+ '" includes:\n'+ ', '.join(movie_director[movie_key][movie_key[1:]]))
            print('\n')
            

Which movie do you want to check? >City lights
What information about this movie do you want to check? (Choose director or cast) >dire
Please enter a valid choice. (Choose director or cast) >directi
Please enter a valid choice. (Choose director or cast) >direct
Please enter a valid choice. (Choose director or cast) >casT

The cast of the movie "city lights" includes:
Buster Brodie, Hank Mann, Virginia Cherrill, Henry Bergman, Eddie Baker, Albert Austin, Victor Alexander, Harry Myers, Jack Alexander, Florence Lee, Betty Blair, T.S. Alexander, Al Ernest Garcia, Harry Ayers, Charles Chaplin




## Then there are 3 tasks you need to complete:

### 1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?


#### directors_list which contains the list of directors, movie wise is looped and a directors_list_individual is formed. this new list directors_list_individual, contains the names of all the directors of 500 movies. In the same loop we create a dictiobary which stores directors name as the key and count of movies directed as the value. This list could be used to analyze the counts of movies directed by the directors like :
#### list of directors who directed more than 6 movies,
#### list of directors who directed less than 6 movies etc.
## we can change the value of the variable 'number_of_movies' and analyze for other scenarios

#### We can calculate the success factor of the directors by calculating the number of movies directed by them, A director is more successful if he/she has directed more movies and viceversa.
#### Total number of director(s) who are in the Top 500 movie list: 411 

In [20]:
# Change the value here and analyze:
number_of_movies=6

directors_list_individual = list()
director_movie_count_dict={}
for directors_sublist in directors_list:
    for director in directors_sublist:
        if director not in directors_list_individual:
            directors_list_individual.append(director)
        director_movie_count_dict[director] = director_movie_count_dict.get(director,0)+1

print("Total number of director(s) who are in the Top 500 movie list:", len(directors_list_individual),"\n")



print("Below are the director(s) who have directed",number_of_movies,"or more than", number_of_movies, "movies:")
i=0
for key, value in director_movie_count_dict.items():
    if value>=number_of_movies:
        i+=1
        print(key , value)
print("\nTotal Count of director(s) who have directed",number_of_movies,"or more than", number_of_movies, "movies:",i)
print(len(directors_list_individual)-i,"directors have directed less than",number_of_movies,"movies")

Total number of director(s) who are in the Top 500 movie list: 411 

Below are the director(s) who have directed 6 or more than 6 movies:
Alfred Hitchcock 12
George Cukor 6
Martin Scorsese 8
Steven Spielberg 6

Total Count of director(s) who have directed 6 or more than 6 movies: 4
407 directors have directed less than 6 movies


## We can do the similar analysis like the director, for the actors as well
#### cast_list and p_cast_list which contains the list of actors/cast, movie wise is looped and a cast_list_individual is formed. this new list cast_list_individual, contains the names of all the casts of 500 movies. In the same loop we create a dictionary which stores cast name as the key and count of movies acted as the value. This list could be used to analyze the counts of movies acted by the actors like :
#### list of actors who acted in more than 5 movies,
#### list of actors who acted in less than 5 movies etc.
## we can change the value of the variable 'number_of_movies' and analyze for other scenarios

#### We can calculate the success factor of the actors by calculating the number of movies acted by them, A actor is more successful if he/she has acted in more movies and viceversa.
#### Total number of cast(s) who are in the Top 500 movie list: 6503

In [21]:
# Change the value here and analyze:
number_of_movies=5

cast_list_individual = list()
cast_movie_count_dict={}
cast_list_unique = []
for i in range(len(movie_names)):
    cast_list_unique.append(list(set(p_cast_list[i]+cast_list[i]))) 
            
for cast_sublist in cast_list_unique:
    for cast in cast_sublist:
        if cast not in cast_list_individual:
            cast_list_individual.append(cast)
        cast_movie_count_dict[cast] = cast_movie_count_dict.get(cast,0)+1

print("Total number of cast(s) who are in the Top 500 movie list:", len(cast_list_individual),"\n")

print("Below are the cast(s) who have acted in",number_of_movies,"or more than", number_of_movies, "movies:")
i=0
for key, value in cast_movie_count_dict.items():
    if value>=number_of_movies:
        i+=1
        print(key , value)
print("\nTotal Count of cast(s) who have acted in",number_of_movies,"or more than", number_of_movies, "movies:",i)
print(len(cast_list_individual)-i,"cast(s) have acted in less than",number_of_movies,"movies")

Total number of cast(s) who are in the Top 500 movie list: 6503 

Below are the cast(s) who have acted in 5 or more than 5 movies:
Orson Welles 6
Joseph Cotten 5
Marlon Brando 6
Robert Duvall 5
Al Pacino 5
James Stewart 6
John Qualen 5
Humphrey Bogart 5
Daniel Day-Lewis 5
Harrison Ford 5
Ward Bond 6
John Ratzenberger 7
Robert De Niro 7
Harvey Keitel 6
Wallace Shawn 5
Ralph Fiennes 6
Timothy Spall 6
Max von Sydow 5
Willem Dafoe 5
Jim Broadbent 5

Total Count of cast(s) who have acted in 5 or more than 5 movies: 20
6483 cast(s) have acted in less than 5 movies


### 2. Create a dictionary of actors/actresses that the directors have worked together with in each movie, then calculate their cosine similarity, which directors work with similar groups of actors/actresses? Use director name as the dictionary name, actor/actress name as the key, and the times they have worked together in a movie as the value. For example: Michael Bay = {‘ Bruce Willis’: 50, ‘Ben Affleck’:20, ‘Liam Neeson’:10}, Steven Spielberg = {‘Liam Neeson’: 30, ‘Tom Hanks’:20, ‘Denzel Washington’:15}
### Your program should show the similarity score between the directors. (An example is given below).


#### A dir_cast_dict dictionary is created where director is the key and the list of actors/actress who have acted in the movie directed by the director are the values. To obtain this, the main dictionary, movie_director is looped and the cast is stored as a list for each of the director, (for all the movies). The structure of the dictionary is as follows:
#### dict = {'Steven Spielberg':{'Tom Hanks':2, 'Liam Neeson':2}
####            'Michael Bay':{'Bruce Willis':2, 'Liam Neeson':1}}

In [23]:
from collections import Counter
dir_cast_dict={}
for i in movie_director.values():
    for dir, cast in i.items():
        for ind_dir in dir:
            dir_cast_dict[ind_dir] = dir_cast_dict.get(ind_dir, [] )+cast
            
for key, value in dir_cast_dict.items():
    dir_cast_dict[key] = dict(Counter(value))
# print(dir_cast_dict)
directors_list_individual_lower = list(map(lambda x:x.lower(), directors_list_individual))

# print(len(directors_list_individual_lower))

## Now run the below cell to find the cosine similarity between the director names provided
####  The user is prompted repeatedly to enter the 2 directors names correctly until the user provides the righ name (case sensitivity is handled)
#### Based on input provided by the user, cosine similarity between these 2 directors is found (with the actors that they have worked together)
#### High similarity will result in a value close to 1 and a low similarity would result in a similarity value of 0.

In [40]:
print("Let's find the cosine similarity. Select any 2 directors and provide the value to proceed:")
director1 = input('Enter director 1 name >').strip().lower()
while True:
    try:
        # Raise an exception if user enters wrong director name (not in the list)
        if director1 not in directors_list_individual_lower:
            raise
        else:
            break
    except:
        director1 = input('Please enter a valid director name. Enter director 1 name >').strip().lower()

director2 = input('Enter director 2 name >').strip().lower()
while True:
    try:
        # Raise an exception if user enters wrong director name (not in the list)
        if director2 not in directors_list_individual_lower:
            raise
        else:
            break
    except:
        director2 = input('Please enter a valid director name. Enter director 2 name >').strip().lower()

        #we first need to extract the list of actors and how many time they have worked together
for k, v in dir_cast_dict.items():
    if k.lower() == director1:
        sub_dic1 = v

for k, v in dir_cast_dict.items():
    if k.lower() == director2:
        sub_dic2 = v
        
#then we need to create the combined list containing all actors that the 2 directors have worked with
#first we extract the actor names, which are the keys
sub_actor1 = list(sub_dic1.keys())
sub_actor2 = list(sub_dic2.keys())

#then we convert them into sets, so that we can find a combined set using the union function
set_actor1 = set(sub_actor1)
set_actor2 = set(sub_actor2)
combine_set = set_actor1.union(set_actor2) 

#let's next create a dictionary out of the combined list, filling in 0 for all the values
# you will see why in the next step
dir1_actor = dict()
for i in combine_set:
    dir1_actor[i] = 0

dir2_actor = dict()
for i in combine_set:
    dir2_actor[i] = 0

#for those actors that director1 have worked with, replace the 0 with the actual values
for key, value in dir1_actor.items():
    for k, v in sub_dic1.items():
        if key == k:
            dir1_actor[key] = v

#do the same thing for director2
for key, value in dir2_actor.items():
    for k, v in sub_dic2.items():
        if key == k:
            dir2_actor[key] = v

#now that we have a uniform dictionary across the 2 directors, we can extract the values in vectors
#ready for comparison
dir1_vector = list(dir1_actor.values())
dir2_vector = list(dir2_actor.values())

#calculate cosine similarity
divident = 0 
for i in range(len(dir1_vector)):
    summing = dir1_vector[i] * dir2_vector[i]
    divident = divident + summing

a = 0
b = 0
for i in range(len(dir1_vector)):
    sum1 = (dir1_vector[i])**2
    a = a + sum1
    sum2 = (dir2_vector[i])**2
    b = b + sum2
divisor = (a**0.5)*(b**0.5)

cosine_similarity = divident/divisor
print("Cosine Similarity for "+director1+" and "+director2+" is:")
print(cosine_similarity)

Let's find the cosine similarity. Select any 2 directors and provide the value to proceed:
Enter director 1 name >Charles Chaplin
Enter director 2 name >Sam Wood
Cosine Similarity for charles chaplin and sam wood is:
0.0


## Sample outputs:
### Scenario 1:
#### Let's find the cosine similarity. Select any 2 directors and provide the value to proceed:
#### Enter director 1 name >Charles Chaplin
#### Enter director 2 name >Charles Chaplin
#### Cosine Similarity for charles chaplin and charles chaplin is:
#### 1.0000000000000002

### Scenario 2:
#### Let's find the cosine similarity. Select any 2 directors and provide the value to proceed:
#### Enter director 1 name >Sam Wood
#### Enter director 2 name >George Cukor
#### Cosine Similarity for sam wood and george cukor is:
#### 0.3952847075210474

### Scenario 3:
#### Let's find the cosine similarity. Select any 2 directors and provide the value to proceed:
#### Enter director 1 name >Charles Chaplin
#### Enter director 2 name >Sam Wood
#### Cosine Similarity for charles chaplin and sam wood is:
#### 0.0

### 3. Pick 5 of your favorite actors/actresses from this list of top 500 movies. Then create a dictionary of all the actors/actresses that they have collaborated with in a movie. Following similar method as above in task 2, create the dictionaries, and compare these 5 actors, who is the most popular supporting actor/actress among them all?

#### 5 of your favorite actors/actresses from this list of top 500 movies are hardcoded/listed below. Please change the values and rerun to find the cosine similarity amongst themselves
#### support_cast_dict dictionary created by looping the movie_director dictionary and for each of the favourite actors in the list we find out the supporting cast. At the end of the loop we get a dictionary which contains favourite actor names as the key and the list of supporting actors as the value
#### Taking the combinations amongst the actors, we find out the cosine similarity between the combinations as follows

In [76]:
from collections import Counter
favourite_actors = [ 'Al Pacino', 'Marlon Brando', 'Orson Welles', 'Anna Paquin','Karl Malden' ]
favourite_actors = list(map(lambda x:x.lower(), favourite_actors))
support_cast_dict={}
for i in movie_director.values():
    for dir, cast in i.items():
        for ind_cast in cast:
            if ind_cast.lower() in favourite_actors:
                support_cast_dict[ind_cast.lower()] = support_cast_dict.get(ind_cast.lower(), [] )+ list(map(lambda x:x.lower(), cast))
# print(support_cast_dict)  
for key, value in support_cast_dict.items():
#     print(key)
    temp_list = list(set(value))
    temp_list.remove(key)
    support_cast_dict[key] = dict(Counter(temp_list))
# print(support_cast_dict)



print('Cosine similarity between the 5 favourite actors are as folllows:')
for i in range(len(support_cast_dict)):
    
    for j in range(i+1,len(support_cast_dict)):
        common_vectors = set()
        cast1_vector = list()
        cast2_vector = list()
        for cast in support_cast_dict[favourite_actors[i]]:
            common_vectors.add(cast)
        
        for cast in support_cast_dict[favourite_actors[j]]:
            common_vectors.add(cast)

        for cast in common_vectors:
            if cast in support_cast_dict[favourite_actors[i]]:
                cast1_vector.append(support_cast_dict[favourite_actors[i]][cast])
            else:
                cast1_vector.append(0)

            if cast in support_cast_dict[favourite_actors[j]]:
                cast2_vector.append(support_cast_dict[favourite_actors[j]][cast])
            else:
                cast2_vector.append(0)
                
        divident = 0 
        for a in range(len(cast1_vector)):
            summing = cast1_vector[a] * cast2_vector[a]
            divident = divident + summing

        a = 0
        b = 0
        for z in range(len(cast1_vector)):
            sum1 = (cast1_vector[z])**2
            a = a + sum1
            sum2 = (cast2_vector[z])**2
            b = b + sum2
        divisor = (a**0.5)*(b**0.5)
        
        
        cosine_similarity = divident/divisor
       
        print(favourite_actors[i]+"\t\t\t"+favourite_actors[j] +"\t\t\t"+str(cosine_similarity))

Cosine similarity between the 5 favourite actors are as folllows:
al pacino			marlon brando			0.2256890580767064
al pacino			orson welles			0.0
al pacino			anna paquin			0.3743922827009819
al pacino			karl malden			0.03163859985841663
marlon brando			orson welles			0.0
marlon brando			anna paquin			0.019418390934515434
marlon brando			karl malden			0.47178209756024825
orson welles			anna paquin			0.0
orson welles			karl malden			0.0
anna paquin			karl malden			0.0


### Following similar method as above in task 2, create the dictionaries, and compare these 5 actors, who is the most popular supporting actor/actress among them all?
#### Change the value of variable 'n' to get top n most popular actors

In [86]:
# Change the vale to get top n most popular actors
n=5
most_popular_cast_list = []
for i in support_cast_dict.values():
    most_popular_cast_list.extend(i)
most_pop_dict = dict(Counter(most_popular_cast_list).most_common(5))
    
print("Top "+str(n)+" most popular supporting actor/actress among them all are:")
for key, value in most_pop_dict.items():
    print(key+"\t acted in \t"+str(value) +"\t movies")


Top 5 most popular supporting actor/actress among them all are:
rudy bond	 acted in 	3	 movies
kim hunter	 acted in 	2	 movies
lee j. cobb	 acted in 	2	 movies
vivien leigh	 acted in 	2	 movies
richard s. castellano	 acted in 	2	 movies


## References:

### Professor's Lecture Video and slides:  on Canvas

https://uta.instructure.com/courses/78408/conferences


https://www.sqlitetutorial.net/sqlite-python/create-tables/

https://realpython.com/python-csv/

https://www.programiz.com/python-programming/writing-csv-files

https://www.geeksforgeeks.org/create-a-tuple-from-string-and-list-python/

https://www.geeksforgeeks.org/python-get-unique-values-list/

https://www.tutorialspoint.com/python/python_tuples.htm

https://www.geeksforgeeks.org/python-count-occurrences-element-list/

